In [4]:
import requests
import pandas as pd
import boto3
import os
from datetime import datetime as dt
from datetime import timedelta as td



In [5]:
dynamodb_resource = boto3.resource('dynamodb')

job_details_table = dynamodb_resource.Table('jobs')


In [6]:
job_details_table.delete_item(Key={'job_id': 'ghactivity_ingest'})


{'ResponseMetadata': {'RequestId': '4311INKIQ0B7NIHQU0Q5D9BDQNVV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Sun, 13 Aug 2023 12:17:14 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': '4311INKIQ0B7NIHQU0Q5D9BDQNVV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '2745614147'},
  'RetryAttempts': 0}}

In [7]:
job_details = {
    'job_id': 'ghactivity_ingest',
    'job_description': 'Ingest ghactivity data to s3',
    'is_active': 'Y',
    'baseline_days':3
}

In [8]:
job_details_table.put_item(Item=job_details)

{'ResponseMetadata': {'RequestId': '3MF1TR9O5CM2EC7PCSFSKMKL3RVV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Sun, 13 Aug 2023 12:17:15 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': '3MF1TR9O5CM2EC7PCSFSKMKL3RVV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '2745614147'},
  'RetryAttempts': 0}}

In [9]:
def get_job_details(job_name):
    dynamodb = boto3.resource('dynamodb')
    table= dynamodb.Table('jobs')
    job_details= table.get_item(Key={'job_id':job_name})['Item']
    return job_details
      

In [10]:
job_details = get_job_details('ghactivity_ingest')
job_details

{'job_description': 'Ingest ghactivity data to s3',
 'is_active': 'Y',
 'job_id': 'ghactivity_ingest',
 'baseline_days': Decimal('3')}

In [11]:
dt.now()

datetime.datetime(2023, 8, 13, 17, 47, 16, 294721)

In [12]:
dt.now().date()

datetime.date(2023, 8, 13)

In [13]:
dt.now().date() - td(days=3)

datetime.date(2023, 8, 10)

In [14]:
next_file = f'{dt.strftime(dt.now().date()-td(days=3),"%Y-%m-%d")}-0.json.gz'
next_file

'2023-08-10-0.json.gz'

In [15]:
res = requests.get(f'https://data.gharchive.org/{next_file}')

In [16]:
file = open(f'data/{next_file}','wb')
file.write(res.content)


75005991

In [17]:
file.close

<function BufferedWriter.close>

In [18]:
!ls -ltr data/

total 106448
-rw-r--r-- 1 admin123 admin123 33989636 Aug  8 17:12 2022-06-05-0.json.gz
-rw-r--r-- 1 admin123 admin123 75005991 Aug 13 17:47 2023-08-10-0.json.gz


In [19]:
df = pd.read_json(f'data/{next_file}',
                  lines = True,
                  orient='records')

In [20]:
df.head()

,id,type,actor,repo,payload,public,created_at,org
0,31017097610,PullRequestReviewEvent,"{'id': 42153618, 'login': 'beatrizmilz', 'disp...","{'id': 628083156, 'name': 'LatinR/latinrweb-v2...","{'action': 'created', 'review': {'id': 1570788...",True,2023-08-10 00:00:00+00:00,"{'id': 42558048, 'login': 'LatinR', 'gravatar_..."
1,31017097613,CreateEvent,"{'id': 86026637, 'login': 'LucioMJ99', 'displa...","{'id': 676754092, 'name': 'LucioMJ99/Bomberman...","{'ref': 'main', 'ref_type': 'branch', 'master_...",True,2023-08-10 00:00:00+00:00,NaN
2,31017097635,PullRequestEvent,"{'id': 49699333, 'login': 'dependabot[bot]', '...","{'id': 657106737, 'name': 'pratikshatiwari/esh...","{'action': 'opened', 'number': 12, 'pull_reque...",True,2023-08-10 00:00:00+00:00,NaN
3,31017097636,CreateEvent,"{'id': 116526513, 'login': 'haidyabdulghani', ...","{'id': 676754094, 'name': 'haidyabdulghani/pro...","{'ref': None, 'ref_type': 'repository', 'maste...",True,2023-08-10 00:00:00+00:00,NaN
4,31017097638,PushEvent,"{'id': 141023122, 'login': 'abiodun078', 'disp...","{'id': 674564789, 'name': 'abiodun078/rockyboi...","{'repository_id': 674564789, 'push_id': 146232...",True,2023-08-10 00:00:00+00:00,NaN


In [22]:
df.count()

id            153562
type          153562
actor         153562
repo          153562
payload       153562
public        153562
created_at    153562
org            37128
dtype: int64

In [24]:
df.columns

Index(['id', 'type', 'actor', 'repo', 'payload', 'public', 'created_at',
       'org'],
      dtype='object')

In [28]:
df.dtypes

id                          int64
type                       object
actor                      object
repo                       object
payload                    object
public                       bool
created_at    datetime64[ns, UTC]
org                        object
dtype: object